In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import re
import geopandas as gpd
import json
from geoband.API import *
GetCompasData('SBJ_2102_001', '2', '2.세종시_표제부.csv')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:91: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
# 카카오맵 API에 키워드를 입력해 위도와 경도를 반환하는 함수입니다.

def getLatLon_word(word, key):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(word)
        headers = {"Authorization": "KakaoAK {}".format(key)}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]
    except:
        return 'No Result', 'No Result'
    else:
        return float(match_first['y']), float(match_first['x']) # 위도(lat), 경도(lon) 순서로 출력
    

# 카카오맵 API에 주소를 입력해 위도와 경도를 반환하는 함수입니다.
    
def getLatLon(addr, key):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
        headers = {"Authorization": "KakaoAK {}".format(key)}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address']
    except:
        return 'No Result', 'No Result'
    else:
        return float(match_first['y']), float(match_first['x']) # 위도(lat), 경도(lon) 순서로 출력

# 본인의 Rest API key 입력 
key = '5f32cecabbc63fe256f82f340cf5417d'

In [ ]:
# 표제부 파일을 읽어옵니다.
poje = pd.read_csv('2.세종시_표제부.csv')

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
poje.head()

,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,146,75,일반,NaN,NaN,135.24,벽돌구조,단독주택,0,1977.0,NaN
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,1동,576,강파이프구조,동.식물관련시설,0,1999.0,NaN
2,3,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,2동,336,강파이프구조,동.식물관련시설,0,20041223.0,NaN
3,4,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,3동,127.19,강파이프구조,창고시설,0,19950726.0,NaN
4,5,세종특별자치시 반곡동 180-1번지,36110,10100,1,180,1,일반,솔빛숲유치원,NaN,2697.74,철근콘크리트구조,교육연구시설,0,20190228.0,NaN


In [ ]:
# Unnamed: 15 열에 값이 오직 하나 존재합니다.
print(poje.shape)
poje.isnull().sum()

(36729, 16)


순번                 0
대지위치               0
시군구코드              0
법정동코드              0
대지구분코드             0
번                  0
지                  0
대장구분코드명            0
건물명            29403
동명칭            20238
연면적(㎡)             0
구조코드명              9
주용도코드명            12
세대수(세대)            0
사용승인일           1212
Unnamed: 15    36728
dtype: int64

In [ ]:
# Unnamed 열에 값이 오직 하나 존재합니다.
poje.loc[10304,:]

순번                                 10305
대지위치           세종특별자치시  조치원읍 서창리 222-5번지
시군구코드                              36110
법정동코드                              25034
대지구분코드                                 1
번                                    222
지                                      5
대장구분코드명                               일반
건물명            조치원읍 서창리 222 단독주택 (고진국) -
동명칭                                    A
연면적(㎡)                           주건축물제1동
구조코드명                             273.35
주용도코드명                          철근콘크리트구조
세대수(세대)                             단독주택
사용승인일                                  0
Unnamed: 15                  2.01308e+07
Name: 10304, dtype: object

In [ ]:
# Unnamed 열에 하나 존재하는 값을 사용승인일 열로 옮깁니다.
poje.loc[10304,'사용승인일']=poje.loc[10304,'Unnamed: 15']

In [ ]:
poje.loc[10304,:]

순번                                 10305
대지위치           세종특별자치시  조치원읍 서창리 222-5번지
시군구코드                              36110
법정동코드                              25034
대지구분코드                                 1
번                                    222
지                                      5
대장구분코드명                               일반
건물명            조치원읍 서창리 222 단독주택 (고진국) -
동명칭                                    A
연면적(㎡)                           주건축물제1동
구조코드명                             273.35
주용도코드명                          철근콘크리트구조
세대수(세대)                             단독주택
사용승인일                        2.01308e+07
Unnamed: 15                  2.01308e+07
Name: 10304, dtype: object

In [ ]:
# Unnamed 열을 삭제합니다.
del poje['Unnamed: 15']

In [ ]:
poje_unique = poje.drop_duplicates(subset=['대지위치','건물명'])
poje_unique['위도']=0
poje_unique['경도']=0

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(len(poje_unique))
poje_unique.index = range(len(poje_unique))
poje_unique

25339


,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,위도,경도
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,146,75,일반,NaN,NaN,135.24,벽돌구조,단독주택,0,1977.0,0,0
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,1동,576,강파이프구조,동.식물관련시설,0,1999.0,0,0
2,5,세종특별자치시 반곡동 180-1번지,36110,10100,1,180,1,일반,솔빛숲유치원,NaN,2697.74,철근콘크리트구조,교육연구시설,0,20190228.0,0,0
3,6,세종특별자치시 반곡동 203-127번지,36110,10100,1,203,127,일반,NaN,NaN,92.71,벽돌구조,단독주택,0,1989.0,0,0
4,7,세종특별자치시 반곡동 203-189번지,36110,10100,1,203,189,일반,NaN,NaN,188.96,철근콘크리트구조,단독주택,0,20010525.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25334,36714,세종특별자치시 소정면 고등리 블록 세종첨단일반산업단지지구 2공구블록 12-2블럭로트,36110,39024,3,0,0,일반,천연흑연 음극재 제2공장,8동(락커동),183.12,철근콘크리트구조,공장,0,20191017.0,0,0
25335,36723,세종특별자치시 소정면 고등리 블록 세종첨단일반산업단지지구 S3-1BL블록,36110,39024,3,0,0,일반,NaN,주건축물제3동,84.93,경량철골구조,제2종근린생활시설,0,20200410.0,0,0
25336,36726,세종특별자치시 소정면 고등리 블록 세종첨단일반산업단지지구 산업6-2블록,36110,39024,3,0,0,일반,이지스 세종공장,주건축물제1동,957.08,일반철골구조,공장,0,20160711.0,0,0
25337,36727,세종특별자치시 소정면 고등리 블록 세종첨단일반산업단지지구 산업9-1블록,36110,39024,3,0,0,일반,NaN,NaN,0,NaN,NaN,0,NaN,0,0


In [ ]:
key = '5f32cecabbc63fe256f82f340cf5417d'

for i in range(len(poje_unique)):
    addr = poje_unique.loc[i, '대지위치']
    word = poje_unique.loc[i, '건물명']
    
    if '번지' in addr:
        addr = re.sub(r'번지','',addr)

        lat, lon = getLatLon(addr, key)
        poje_unique.loc[i, '위도']=lat
        poje_unique.loc[i, '경도']=lon
        
    elif '블록' in addr:
        word = '세종 ' + str(word)

        lat, lon = getLatLon_word(word, key)
        poje_unique.loc[i, '위도']=lat
        poje_unique.loc[i, '경도']=lon
        
    if i % 500 == 0 :
        print('{}/{} Completed'.format(i, len(poje_unique)))

/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


0/25339 Completed
500/25339 Completed
1000/25339 Completed
1500/25339 Completed
2000/25339 Completed
2500/25339 Completed
3000/25339 Completed
3500/25339 Completed
4000/25339 Completed
4500/25339 Completed
5000/25339 Completed
5500/25339 Completed
6000/25339 Completed
6500/25339 Completed
7000/25339 Completed
7500/25339 Completed
8000/25339 Completed
8500/25339 Completed
9000/25339 Completed
9500/25339 Completed
10000/25339 Completed
10500/25339 Completed
11000/25339 Completed
11500/25339 Completed
12000/25339 Completed
12500/25339 Completed
13000/25339 Completed
13500/25339 Completed
14000/25339 Completed
14500/25339 Completed
15000/25339 Completed
15500/25339 Completed
16000/25339 Completed
16500/25339 Completed
17000/25339 Completed
17500/25339 Completed
18000/25339 Completed
18500/25339 Completed
19000/25339 Completed
19500/25339 Completed
20000/25339 Completed
20500/25339 Completed
21000/25339 Completed
21500/25339 Completed
22000/25339 Completed
22500/25339 Completed
23000/25339 

In [ ]:
for i in poje_unique[poje_unique.위도=='No Result'].index:
    addr = poje_unique.loc[i, '대지위치']
    word = poje_unique.loc[i, '건물명']
    
    if '번지' in addr:
        word = '세종 '+str(word)
        lat, lon = getLatLon_word(word, key)
        poje_unique.loc[i, '위도']=lat
        poje_unique.loc[i, '경도']=lon

/opt/app-root/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
poje_save = poje.merge(poje_unique[['대지위치','위도','경도']],right_on='대지위치',left_on='대지위치')

In [ ]:
poje_save.to_csv('전처리_2.세종시_표제부.csv', index=False, encoding='CP949')